In [ ]:
pip install konlpy

In [ ]:
string = []
tag = []
with open("./drive/My Drive/ratings_train.txt",'r',encoding='utf-8-sig') as f:
    cnt = 0
    for line in f.readlines():
        if cnt == 0:
            pass
        else:
            line = line.split('\t')
            string.append(line[1])
            tag.append(int(line[2][0]))
        cnt += 1

from konlpy.tag import Okt
twitter = Okt()

string_morphs = []
string_char = []

max_morph_len = 0

morph_set = set()

In [ ]:

from keras.preprocessing.sequence import pad_sequences
cnt = 0
string_morphs = []
for line in string:
    morph = twitter.morphs(line)
    max_morph_len = max_morph_len if max_morph_len > len(morph) else len(morph)
    string_morphs.append(morph)
    morph_set.update(morph)

input_morph = []

morph_set = list(['<p>']) + list(morph_set)
morph_set = {mor:index for index,mor in enumerate(morph_set)}

In [ ]:
for line in string_morphs:
    morph = [morph_set[mor] for mor in line]
    input_morph.append(morph)
input_morph = pad_sequences(input_morph ,padding = 'post'
    , maxlen = max_morph_len)

from keras.utils import to_categorical
tag = to_categorical(tag)
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
input_morph_train,input_morph_val,tag_train,tag_val = train_test_split(input_morph,tag,test_size = 0.1,random_state = 1)

In [ ]:
from keras import backend as K
def recall(y_target, y_pred):
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    count_true_positive_false_negative = K.sum(y_target_yn)

    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall


def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
from keras.layers import Input, Dense, LSTM,Conv1D,Embedding,Reshape,GlobalMaxPooling2D,Flatten
from keras.models import Model
input_morph = Input((max_morph_len,))
embed_output = Embedding(len(morph_set),
                         100)(input_morph)
lstm_output = LSTM(128,return_sequences = True)(embed_output)
embed_output = Reshape((max_morph_len,128,1))(lstm_output)
cnn_output = Conv1D(32,kernel_size = 3,activation = 'relu')(embed_output)
cnn_pool = GlobalMaxPooling1D(data_format='channels_last')(cnn_output)
output = Dense(2,activation = 'softmax')(cnn_pool)
model = Model([input_morph],output)
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy',f1score])

In [ ]:
import keras
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=1)]

model.fit(input_morph_train,tag_train,
         epochs = 10,
         callbacks = callbacks,
         batch_size = 256,
         validation_data = (input_morph_val,tag_val))

In [ ]:
test_strings = []
test_labels = []
with open('./drive/My Drive/ratings_test.txt','r',encoding='utf-8-sig') as f:
    cnt = 0
    for line in f.readlines():
        if cnt ==0:
            pass
    else:
        line = line.split('\t')
        test_label = int(line[2][0])
        test_strings.append(line[1])
        test_labels.append(test_label)
    cnt +=1
test_morph = []
test_char = []
for line in test_strings:
#   morph = ''
#   for mor in line:
#     if mor not in stop_word:
#       morph += mor
    morph = twitter.morphs(line)
    morph = [morph_set[mor] if mor in morph_set else 0 for mor in morph]
    test_morph.append(morph)

test_morph = pad_sequences(test_morph,padding = 'post',maxlen = max_morph_len)
# test_char = pad_sequences(test_char,padding = 'post',maxlen = max_char_len)

test_label = to_categorical(test_labels)


In [ ]:
from sklearn.metrics import f1_score 
point = model.evaluate([test_morph],test_labels)
predicted = np.argmax(model.predict(test_morph),axis = 1)
f1_score(predicted,test_labels)